# Jacobi Transformation of a Symmetric Matrix
### Christina Lee
#### based on Numerical Recipes in C++, Sec 11.1

Start with a base Rotation Matrix of the Form
\begin{equation}
    P_{pq} =
    \begin{pmatrix}
           1& &  &  && && 0\\
           & \ddots &&&&  & \\
            && c & \cdots & s && \\
                &&\vdots& 1 & \vdots &&\\
               && -s & \cdots & c && \\
               &&&&& \ddots & \\
               0&&& &  && 1 
    \end{pmatrix}
\end{equation}

From our starting arbitrary symmetric A,
\begin{equation}
A^{T} = A
\end{equation}
we will run a series of transformations,
\begin{equation}
A^{\prime}= P^{T}_{pq} \cdot A \cdot P_{pq}
\end{equation}
where each iteration brings A closer to diagonal form.  Thus in our implementing our algorithm, we need to determine two things
<ul>
<li> The values of c and s
<li> The pattern of sweeping p and q
</ul>
And in the end we will need to finally determine if this actually converges, and if has any sort of efficiency.

So lets expand one transformation, and we if we can solve for $c$ and $s$.

\begin{align}
a^{\prime}_{rp} & = c a_{rp} - s a_{rq} \\
a^{\prime}_{rq} & = c a_{rq} + s a_{rp} \\ 
a^{\prime}_{pp} & = c^2 a_{pp} + s^2 a_{qq} -2 sc a_{pq} \\
a^{\prime}_{qq} & = s^2 a_{qq} + c^2 a_{qq} + 2sc a_{pq} \\
a^{\prime}_{pq} & = \left( c^2-s^2 \right) a_{pq} + sc \left(a_{pq} - a_{qq} \right)
\end{align}

## Determining $s$ and $c$
Given we specifically want $a^{\prime}_{pq}$ to be zero, we re-arrange the last equation,
\begin{equation}
        \frac{c^2-s^2}{2 sc} = \frac{a_{pq}-a_{qq}}{2 a_{pq}} =\theta
\end{equation}
At first glance, this equation might not look easier to solve for $s$ or $c$.  Second either. We define a new parameter $t = s/c$, which now makes the equation,
\begin{equation}
\frac{1-t^2}{2 t} = \theta \;\;\;\; \implies \;\;\; t^2 -2 \theta t -1=0,
\end{equation}
now quite easily solvable by our friendly quadratic formula.  Though the book does recommend using form that pulls out smaller root through
\begin{equation}
t=\frac{\text{sgn}(\theta)}{|\theta| + \sqrt{\theta^2 + 1} }.
\end{equation}
Then reverse solve back to
\begin{align}
c&=\frac{1}{\sqrt{t^2+1}}\\
s&=tc
\end{align}

Though we could use the expressions above, if we simplify them with our new expressions for $c$ and $s$ analytically, we reduce computational load and round off error. These new expressions are
\begin{align}
a^{\prime}_{pq} & = 0\\
a^{\prime}_{qq} & = a_{qq} + t a_{qp} \\
a^{\prime}_{pp} &= a_{pp} - t a_{pq} \\
a^{\prime}_{rp} &= a_{rp} - s \left( a_{rq} +\tau a_{rp} \right) \\
a^{\prime}_{rq} &= a_{rq} + s \left( a_{rp} -\tau a_{rq} \right)
\end{align}
with the new variable
\begin{equation}
\tau = \frac{s}{1+c}
\end{equation}

## Convergence

The sum of the squares of the off diagonal elements (choosen in either upper or lower triagnles arbitrarily)
\begin{equation}
S=\sum\limits_{r < s} |a_{rs}|^2
\end{equation}

## Eigenvectors

By forming a product of every rotation matrix, we also come to approximate the matrix $V$ where 
\begin{equation}
D = V^{T} \cdot A \cdot V
\end{equation}
and $D$ is the diagonal form of $A$.  $V$ is computed through itereative computation
\begin{align}
V^{\prime} & = V \cdot P_i \\
v^{\prime}_{rs} &= v_{rs} \\
v^{\prime}_{rp} &= c v_{rp} - s v_{rq} \\
v^{\prime}_{rq} &= s v_{rp} + c v_{rq} 
\end{align}

### Enough with the talking! LETS COMPUTE STUFF

In [21]:
# First, Lets make our nice, helpful functions

## A function to look at the convergence
function convergence(A::Array)
    num=0.0
    l=size(A)[1]
    for ii in 1:(l-1)
        for jj in (ii+1):l ## just looking at the lower triangle
            num+=A[ii,jj]^2
            #println(ii,' ',jj,' ',num,' ',A[ii,jj])
        end
    end
    return num
end

convergence (generic function with 1 method)

In [97]:
function roundmatrix(A::Array,rtol::Real)
    Ap=copy(A)
    for ii in 1:length(A)
        if abs(Ap[ii])<rtol 
            Ap[ii]=0
        end
    end
    return Ap;
end

roundmatrix (generic function with 1 method)

In [30]:
#Now on to the Rotations!

# We don't always want to compute the eigenvectors, so those are in the 
# optional entries slot. 
# Both tell the function to compute the vectors with computeV=true
# and input the V=V after the semicolon.

function Rotate(A::Array,p::Int,q::Int; computeV=false, V::Array=eye(1))
    θ=(A[q,q]-A[p,p])/(2*A[p,q]);
    t=sign(θ)/(abs(θ)+sqrt(θ^2+1));
    
    c=1/sqrt(t^2+1)
    s=t*c
    τ=s/(1+c)
    
    l=size(A)[1]
    Ap=copy(A[:,p])
    Aq=copy(A[:,q])
    for r in 1:l
        A[r,p]=Ap[r]-s*(Aq[r]+τ*Ap[r]) 
        A[r,q]=Aq[r]+s*(Ap[r]-τ*Aq[r])
        
        A[p,r]=A[r,p]
        A[q,r]=A[r,q]
    end
    A[p,q]=0
    A[q,p]=0
    A[p,p]=Ap[p]-t*Aq[p]
    A[q,q]=Aq[q]+t*Aq[p]
    
    if computeV==true
        Vp=copy(V[:,p])
        Vq=copy(V[:,q])    
        for r in 1:l   
            V[r,p]=c*Vp[r]-s*Vq[r]
            V[r,q]=s*Vp[r]+c*Vq[r]
        end
        return A,V
    else
        return A;
    end
end

Rotate (generic function with 1 method)

In [98]:
n=6;
A=rand(n,n);
for ii in 1:n
    A[ii,1:ii]=transpose(A[1:ii,ii]) 
end
V=eye(n)
A0=copy(A)
A

6x6 Array{Float64,2}:
 0.793162  0.704718  0.57717    0.593039   0.267469  0.75788 
 0.704718  0.853092  0.266642   0.510272   0.36707   0.937841
 0.57717   0.266642  0.578512   0.0683895  0.930363  0.219291
 0.593039  0.510272  0.0683895  0.727548   0.801134  0.670543
 0.267469  0.36707   0.930363   0.801134   0.712329  0.140053
 0.75788   0.937841  0.219291   0.670543   0.140053  0.541262

In [113]:
## keep evaluating for a couple iterations
for ii in 2:n
    for jj in 1:(ii-1)
        if abs(A[ii,jj])>.001
            A,V=Rotate(A,ii,jj;computeV=true,V=V);
        end    
    end
end
roundmatrix(A,1e-4),A,V,convergence(A)

(
6x6 Array{Float64,2}:
 -0.687616     0.000333233   0.0          …  -0.000175403   0.0        
  0.000333233  3.34904       0.0              0.0           0.000148592
  0.0          0.0           1.15674          0.0           0.0        
  0.0          0.0          -0.000364856      0.0           0.000349946
 -0.000175403  0.0           0.0              0.534951      0.0        
  0.0          0.000148592   0.0          …   0.0          -0.299853   ,

6x6 Array{Float64,2}:
 -0.687616      0.000333233   5.12382e-7   …  -0.000175403   3.58693e-5 
  0.000333233   3.34904      -7.19832e-5      -2.576e-7      0.000148592
  5.12382e-7   -7.19832e-5    1.15674         -8.65767e-6    5.66138e-8 
  0.0           2.85225e-5   -0.000364856      4.53029e-7    0.000349946
 -0.000175403  -2.576e-7     -8.65767e-6       0.534951      0.0        
  3.58693e-5    0.000148592   5.66138e-8   …   0.0          -0.299853   ,

6x6 Array{Float64,2}:
  0.227326  0.455959  -0.165182     0.656925   -0.378279  

In [114]:
roundmatrix(V*A*transpose(V)-A0,1e-6)

6x6 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0